# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [16]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [17]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [18]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [19]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [20]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [24]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [26]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [28]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [32]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [33]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [37]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
# Zero shot prompt
context_user = [
    {'role':'system', 'content':
     """You are an expert on equestrian topics and your job is to answer questions for horse owners on an online forum.
     """}
]
print(return_OAIResponse("Should I let my horse go barefoot or are normal horseshoes better?", context_user))

Whether to keep your horse barefoot or use horseshoes ultimately depends on several factors such as your horse's individual needs, the terrain they are in, their workload, and any existing hoof issues.

Barefoot horses can be beneficial as it allows their hooves to naturally expand and contract, promoting better circulation and overall hoof health. It can also help improve their balance and coordination. However, not all horses can go barefoot comfortably, especially if they have brittle hooves or are working on rough, hard surfaces.

If your horse is experiencing hoof issues or discomfort, using horseshoes may provide additional support and protection. Horseshoes can help prevent excessive wear and tear on the hooves, especially for horses in heavy work or those with hoof conformation issues.

It's best to consult with your farrier or veterinarian to determine the best option for your horse based on their individual needs and circumstances. They can assess your horse's hooves and prov

In [22]:
print(return_OAIResponse("Help! My horse is feeling sick! I am scared that it might have eaten a poisonous plant. What must I do?", context_user))

If you suspect that your horse may have ingested a poisonous plant, it is important to act quickly and seek veterinary assistance immediately. Some common signs of plant poisoning in horses include colic, diarrhea, difficulty breathing, staggering, trembling, or seizures. 

In the meantime, try to remove your horse from the pasture or area where the suspected plant is located to prevent further ingestion. Offer clean, fresh water and do not give your horse any food until a veterinarian can evaluate the situation. 

When you contact the vet, provide as much information as you can about the plant in question, including its appearance and any recent changes in your horse's behavior or health. The vet may recommend specific treatment based on the type of plant that was ingested. Immediate veterinary care is crucial in cases of plant poisoning, so do not hesitate to seek help as soon as possible.


In [23]:
# Few shot prompt
context_user = [
    {'role':'system', 'content':
     """You are a primary school teacher, creating questions for a math test.
     Question 1: 4+5
     Answer: 9

     Question 2: 10 x 3
     Answer: 30

    Question 3: 8 / 2
    Answer: 4
     """}
]
print(return_OAIResponse("Please come up with 10 questions", context_user))

Question 4: If you have 3 apples and you eat 2, how many apples do you have left?
Answer: 1

Question 5: What is the next number in this sequence: 2, 4, 6, 8, __
Answer: 10

Question 6: If a rectangle has a length of 5 cm and a width of 3 cm, what is its area?
Answer: 15 square cm

Question 7: How many sides does a triangle have?
Answer: 3

Question 8: If you add 7 to a number and then subtract 3, the result is 12. What is the original number?
Answer: 8

Question 9: What is the value of 5 squared?
Answer: 25

Question 10: If a box contains 20 pencils and you take out 5, how many pencils are left in the box?
Answer: 15


In [24]:
# Few shot prompt, revised
context_user = [
    {'role':'system', 'content':
     """You are a primary school teacher, creating questions for a math test.
     Question 1: 4+5
     Answer: 9

     Question 2: 10 x 3
     Answer: 30

    Question 3: 8 / 2
    Answer: 4
     """}
]
print(return_OAIResponse("Please come up with 10 more questions, which are similar to the 3 examples provided ", context_user))

Question 4: 7 + 3

Question 5: 6 x 4

Question 6: 15 / 3

Question 7: 9 - 2

Question 8: 12 + 8

Question 9: 5 x 7

Question 10: 20 / 4

Question 11: 14 - 6

Question 12: 3 + 9

Question 13: 11 x 2


In [25]:
# Few shot prompt - role "assistant"
context_user = [
    {'role':'assistant', 'content':
     """You are a primary school teacher, creating questions for a math test.
     Question 1: 4+5
     Answer: 9

     Question 2: 10 x 3
     Answer: 30

    Question 3: 8 / 2
    Answer: 4
     """}
]
print(return_OAIResponse("Please come up with 10 more questions like the examples", context_user))

Question 4: 7 - 3    
Answer: 4

Question 5: 6 x 2    
Answer: 12

Question 6: 15 / 3    
Answer: 5

Question 7: 9 + 6    
Answer: 15

Question 8: 20 - 13     
Answer: 7

Question 9: 4 x 4    
Answer: 16

Question 10: 18 / 2    
Answer: 9

Question 11: 12 + 3    
Answer: 15

Question 12: 50 - 20    
Answer: 30

Question 13: 9 x 3    
Answer: 27


Observations:
- I tried a more creative example of an equestrian expert. Without giving many instructions, the model was able to give great answers
- I tried an example of a maths teacher setting up a test. Here, the model did not perform so well. I gave 3 example questions and asked for 10. The model only gave the remaining 7 instead of 10 more and the aquestions did not match the examples provided at all
- When I asked for the questions to be similar to the examples, the model only gave me the questions and not "Question: ... Answer: ..." as in the examples
- When I changed the role from "system" to "assistant", the answer was perfect
- I learnt that on more specific tasks, we have to be more specific in the prompts as well
- Also, the role makes a difference in how the model "thinks" and answers